# TP2 - Bayes Inference

In [ ]:
# @title Helper Functions 
import numpy as np
import matplotlib.pyplot as plt

def my_gaussian(x_vector,mu,sigma):
    """
  Author: Antoine de Comite 
  This function computes the gaussian distribution characterised by mu & sigma on the set x_vector
  
  Inputs : x_vector (numpy array) the set over which you want to compute the gaussian distribution
           mu (double) mean value of the gaussian distribution
           sigma (double) standard deviation of the gaussian distribution
  Outputs: px (numpy array) is the gaussian distribution evaluated over x_vector
  """
  
    px = np.exp(- 1/2/sigma**2 * (mu - x_vector) ** 2)
    px = px / px.sum()
    return px

## Tutorial Objectives

At the end of this tutorial you should be able to : 

1. Understand and implement Bayes' theorem to compute the posterior distribution
2. Use Bayes' theorem to infer a human's decision


# Section 1 - Bayes' Theorem and the Posterior

Bayes' rule states:

\begin{eqnarray}
\text{Posterior} = \dfrac{ \text{Likelihood} \times \text{Prior}}{ \text{Normalization constant}}
\end{eqnarray}

When both the prior and likelihood are Gaussians, this translates into the following form:

$$
\begin{array}{rcl}
\text{Likelihood} &=& \mathcal{N}(\mu_{likelihood},\sigma_{likelihood}^2) \\
\text{Prior} &=& \mathcal{N}(\mu_{prior},\sigma_{prior}^2) \\
\text{Posterior} &\propto& \mathcal{N}(\mu_{likelihood},\sigma_{likelihood}^2) \times \mathcal{N}(\mu_{prior},\sigma_{prior}^2) \\
&&= \mathcal{N}\left( \dfrac{\sigma^2_{likelihood}\mu_{prior}+\sigma^2_{prior}\mu_{likelihood}}{\sigma^2_{likelihood}+\sigma^2_{prior}}, \dfrac{\sigma^2_{likelihood}\sigma^2_{prior}}{\sigma^2_{likelihood}+\sigma^2_{prior}} \right) 
\end{array}
$$

In these equations, $\mathcal{N}(\mu,\sigma^2)$ denotes a Gaussian distribution with parameters $\mu$ and $\sigma^2$:
$$
\mathcal{N}(\mu, \sigma) = \frac{1}{\sqrt{2 \pi \sigma^2}} \; \exp \bigg( \frac{-(x-\mu)^2}{2\sigma^2} \bigg)
$$
 

## Exercise 1A: Finding the posterior computationally

Imagine an experiment where participants estimate the location of tennis ball. To estimate its position, they can use two sources of information: 
  1. New noisy visual information (the likelihood)
  2. Prior expectations of where the ball is likely to be found (prior). 

Use Bayes' rule to combine the likelihood (noisy visual information) and the prior to compute the posterior distribution.

Hint: 

* Use the gaussian function my_gaussian() you wrote in the previous tutorial (we already made it available for you in this notebook) to generate a visual likelihood with parameters $\mu$ = 3 and $\sigma$ = 1.5
* Generate a Gaussian prior with parameters $\mu$ = -1 and $\sigma$ = 1.5
* Calculate the posterior using pointwise multiplication of the likelihood and prior. Don't forget to normalize so the posterior adds up to 1. **Question : Why do we need to normalize?**



In [ ]:
x_vector = np.arange(-10,10,0.1)

def compute_posterior_pointwise(prior, likelihood):
    ''' Author: Florence Blondiaux
    Returns the normalized posterior probability based on the prior and the likelihood
    Prior: The prior probabilities
    Likelihood: The likelihood probabilities
    '''
    ##########################
    ##### Your code here #####
    ##########################


##########################
##### Your code here #####
##########################
# Plot the three distributions


Click [here](https://github.com/fblondiaux/LGBIO2060-2020/blob/master/Solutions/Tp2-Ex1.py) for solution

*Example output:*

<img alt='Solution hint' align='left' width=573 height=416 src=https://raw.githubusercontent.com/fblondiaux/LGBIO2060-2020/master/Solutions/TP2_Ex1.png>

In [ ]:
#@title
#@markdown Make sure you execute this cell to enable the widget!

x = np.arange(-15, 15, 0.1)

import ipywidgets as widgets

def refresh(mu_likelihood=3, sigma_likelihood=1.5, mu_prior=-1, sigma_prior=1.5):
    likelihood = my_gaussian(x, mu_likelihood, sigma_likelihood)
    prior = my_gaussian(x, mu_prior, sigma_prior)
    posterior = compute_posterior_pointwise(prior,likelihood)
    plt.plot(x, likelihood, 'r')
    plt.plot(x,prior,'g')
    plt.plot(x, posterior, 'b')

style = {'description_width': 'initial'}

_ = widgets.interact(refresh,
    mu_posterior=widgets.FloatSlider(value=3, min=-10, max=10, step=0.5, description="mu_likelihood:", style=style),
    sigma_posterior=widgets.FloatSlider(value=1.5, min=0.5, max=10, step=0.5, description="sigma_likelihood:", style=style),
    mu_prior=widgets.FloatSlider(value=-1, min=-10, max=10, step=0.5, description="mu_prior:", style=style),
    sigma_prior=widgets.FloatSlider(value=1.5, min=0.5, max=10, step=0.5, description="sigma_prior:", style=style)
)

## Exercise 1B: Finding the posterior analytically

**[If you are running short on time, feel free to keep the coding exercise below for later].**

The product of two Gaussian distributions, such as our prior and likelihood, remains a Gaussian, regardless of the parameters (**you can verify this property by hand**). We can directly compute the  parameters of that Gaussian from the means and variances of the prior and likelihood. For example, the posterior mean is given by:

$$ \mu_{posterior} = \frac{\mu_{likelihood} \cdot \frac{1}{\sigma_{likelihood}^2} + \mu_{prior} \cdot \frac{1}{\sigma_{prior}^2}}{1/\sigma_{likelihood}^2 + 1/\sigma_{prior}^2} 
$$

This expression is a special case for two Gaussians, but is a very useful one because:


*   The posterior has the same distribution as both the prior and the likelihood
*   There is simple, closed-form expression for its parameters (mean and variance)

When these properties hold, we call them **conjugate distributions** or **conjugate priors** (for a particular likelihood). Working with conjugate distributions is very convenient; otherwise, it is often necessary to use computationally-intensive numerical methods to combine the prior and likelihood. 

In this exercise, we ask you to verify that property.  To do so, we will hold our likelihood constant as an $\mathcal{N}(3, 1.5)$ distribution, while considering priors with different means ranging from $\mu=-10$ to $\mu=10$. For each prior,

* Compute the posterior distribution using the function you wrote in Exercise 1A. Next, find its mean. The mean of a probability distribution is $\int_x p(x) dx$ or $\sum_x x\cdot p(x)$. 
* Compute the analytical posterior mean from likelihood and prior using the equation above.
* Plot both estimates of the mean. 

Are the estimates of the posterior mean the same in both cases? 

Using these results, try to predict the posterior mean for the combination of a $\mathcal{N}(-4,4)$ prior and and $\mathcal{N}(4, 2)$ likelihood. Use the widget above to check your prediction. You can enter values directly by clicking on the numbers to the right of each slider; $\sqrt{2} \approx 1.41$.

In [ ]:
def compare_computational_analytical_means():
    ##########################
    ##### Your code here #####
    ##########################
    return mu_priors, mus_analytical, mus_by_integration

mu_visuals, mu_analytical, mu_computational = compare_computational_analytical_means()
##########################
##### Your code here #####
##########################
#Visualize the results


Click [here](https://github.com/fblondiaux/LGBIO2060-2020/blob/master/Solutions/Tp2-Ex2.py) for solution

*Example output:*

<img alt='Solution hint' align='left' width=573 height=416 src=https://raw.githubusercontent.com/fblondiaux/LGBIO2060-2020/master/Solutions/TP2_Ex2.png>

# Section 2 - Application of Bayes' theorem

---
  
We'll now estimate the participant response given the posterior distribution. We will describe all the steps of the generative model first. The model will be the same Bayesian model we have been using during the previous exercice: a Gaussian prior and a Gaussian likelihood.

Steps:

* First, we'll create the prior, likelihood, posterior, etc in a form that will make it easier for us to visualise what is being computed and estimated at each step of the generative model: 
  1. Generating a Gaussian prior for multiple possible stimulus inputs
  2. Generating the likelihood for multiple possible stimulus inputs
  3. Estimating our posterior as a function of the stimulus input
  4. Estimating a participant response given the posterior
  


![Generative model](https://raw.githubusercontent.com/fblondiaux/LGBIO2060-2020/master/Solutions/TP2-GenerativeModel.png)


Here is a graphical representation of the generative model:

  1. We present a stimulus $x$ to participants. 
  2. The brain encodes this true stimulus $x$ noisily (this is the brain's representation of the true visual stimulus: $p(\tilde x|x)$.
  3. The brain then combine this brain encoded stimulus (likelihood: $p(\tilde x|x)$) with prior information (the prior: $p(x)$) to make up the brain's estimated position of the true visual stimulus, the posterior: $p(x|\tilde x)$. 
  3. This brain's estimated stimulus position: $p(x|\tilde x)$, is then used to make a response:  $\hat x$, which is the participant's noisy estimate of the stimulus position (the participant's percept). 
  
Typically the response $\hat x$ also includes some motor noise (noise due to the hand/arm move being not 100% accurate), but we'll ignore it in this tutorial and assume there is no motor noise.

We will use the same experimental setup as in the previous exercice. Our subject tries to locates his glasses on his bed stand.

---
# 1 -  Likelihood array
    
The first step is to create the likelihood distribution from the encoded stimulus. For the sake of visualization, we will create a likelihood $f(x) = p(\tilde x|x)$ for each potential encoded stimulus $\tilde x$. We will then be able to visualize the likelihood as a function of hypothesized true stimulus positions: $x$ on the x-axis and encoded position $\tilde x$ on the y-axis.

**Exercise 2.A**
  Using the equation for the `my_gaussian` and the values in `hypothetical_stim`:
  
* Create a Gaussian likelihood with mean varying from `hypothetical_stim`, keeping $\sigma_{likelihood}$ constant at 1.
* Each likelihood will have a different mean and thus a different row-likelihood of your 2D array, such that you end up with a likelihood array made up of 1,000 row-Gaussians with different means. (_Hint_: `np.tile` won't work here. You may need a for-loop).
* Plot the array using the function `plot_myarray()` already pre-written and commented-out in your script

In [ ]:
x = np.arange(-10, 10, 0.1)
hypothetical_stim = np.linspace(-8, 8, 1000)

def compute_likelihood_array(x_points, stim_array, sigma=1.):
    """
    This function computes the likelihood array of a given stimulus
    
    Inputs : x_points (numpy array) is the set of points on which one wants to evaluate the likelihood
             stim_array (numpy array) contains the mean position of every possible stimulus
             sigma (float) is the standard deviation of the gaussian distribution
             
    Outputs : likelihood array (numpy array) is a len(stim_array) x len(x_points) that contains the evaluation of 
              the likelihood for every possible stimulus on the the set x_points. Each line of this array corresponds
              to a single simulus (stim_array[i]).
    """
    #########################
    ## your code goes here ##
    #########################
    return likelihood_array 


# Run the line below to test your code
likelihood_array = compute_likelihood_array(x, hypothetical_stim)

# Plot the results
    #########################
    ## your code goes here ##
    #########################


Click [here](https://github.com/fblondiaux/LGBIO2060-2020/blob/master/Solutions/Tp2-Ex3.py) for solution

*Example output:*

<img alt='Solution hint' align='left' width=573 height=416 src=https://raw.githubusercontent.com/fblondiaux/LGBIO2060-2020/master/Solutions/TP2_Likelihood.png>

**Represents the likelihood distribution for a given stimulus encoding**

---
# 2 - Mixture-of-Gaussians Prior

The prior distribution can sometimes be more complex than a single Gaussian distribution. In this exercise, we will implement a prior distribution which is a mixture of gaussian distributions. To go back to our tennis example, let us assume that your opponent sends the ball in the middle of the court 90% of the time and in the corner of the court 10% of the time. A mixture of Gaussian will combine these two information to obtain a prior that integrate both information. 

Here, we will combine those into a mixure-of-Gaussians probability density function (PDF) that captures both possibilties. We will control how the Gaussians are mixed by summing them together with a 'mixing' or weight parameter $p_{centre}$, set to a value between 0 and 1, like so:

\begin{eqnarray}
\text{Mixture} = \left(p_{\text{centre}} \times \mathcal{N}\left(\mu_{\text{centre}},\sigma_{\text{centre}}\right)\right) + \left(\left(1-p_{\text{centre}}\right) \times \mathcal{N}\left(\mu_{\text{corner}},\sigma_{\text{corner}}\right)\right)
\end{eqnarray}

where $p_{centre}$ denotes the probability that the ball lands at the centre of the courrt.



For visualization reasons, we will create a prior that has the same shape (form) as the likelihood array we created in the previous exercise. That is, we want to create a mixture of Gaussian prior as a function the the brain encoded stimulus $\tilde x$. Since the prior does not change as a function of $\tilde x$ it will be identical for each row of the prior 2D array. 

Using the equation for the Gaussian `my_gaussian`:
* Generate a Gaussian $center$ with mean 0 and standard deviation 0.5. 
* Generate another Gaussian $corner$ with mean 3 and standard deviation 1
* Combine the two Gaussians (center + corner) to make a new prior by mixing the two Gaussians with mixing parameter $p_{corner}$ = 0.50. Make it such that the peakier Gaussian has 50% of the weight (don't forget to normalize afterwards)
* This will be the first row of your prior 2D array
* Now repeat this for varying brain encodings $\tilde x$. Since the prior does not depend on $\tilde x$ you can just repeat the prior for each $\tilde x$ (hint: use np.tile) that row prior to make an array of 1,000 (i.e. `hypothetical_stim.shape[0]`)  row-priors.
* Plot the matrix using the function `plot_myarray()` already pre-written and commented-out in your script


In [ ]:
x = np.arange(-10, 10, 0.1)

def calculate_prior_array(x_points, stim_array, p_center,
                          prior_mean_center=.0, prior_sigma_center=.5,
                          prior_mean_corner=3, prior_sigma_corner=1):
    """
    This function computes the prior distribution based on the mixture of gaussians
    
    Inputs : x_points (numpy array) is the set of points on which we want to evaluate the prior
             stim_array (numpy array) is the array of stimulus input
             p_center (float) is the probability that the ball lands in center of the court
             mean_center (float) mean value of the center ball landing site
             mean_corner (float) mean value of the corner ball landing site 
             sigma_center (float) std deviation of the center ball landing site
             sigma_corner (float) std deviation of the corner ball landing site 
             
    Outputs : prior array (numpy array) contains the prior in a len(stim_array) x len(x_points) array
        'indep' stands for independent
    """

    ###########################
    ### your code goes here ###
    ###########################
   
    return prior_array


# Run the lines below to test your code


p_center=.5
hypothetical_stim = np.linspace(-8, 8, 1000)
prior_array = calculate_prior_array(x, hypothetical_stim, p_center)

    ###########################
    ### your code goes here ###
    ###########################


Click [here](https://github.com/fblondiaux/LGBIO2060-2020/blob/master/Solutions/Tp2-Ex4.py) for solution

*Example output:*

<img alt='Solution hint' align='left' width=573 height=416 src=https://raw.githubusercontent.com/fblondiaux/LGBIO2060-2020/master/Solutions/TP2_Prior.png>

---
# Section 3: Bayes Theorem with Complex Posteriors

We now want to compute the posterior distribution using *Bayes Rule*. Since we have already created a likelihood and a prior for each brain encoded position $\tilde x$, all we need to do is to multiply them row-wise. That is, each row of the posterior array will be the posterior resulting from the multiplication of the prior and likelihood of the same equivalent row.

Mathematically:

\begin{eqnarray}
    Posterior\left[i, :\right] \propto Likelihood\left[i, :\right] \odot Prior\left[i, :\right]
\end{eqnarray}

where $\odot$ represents the [Hadamard Product](https://en.wikipedia.org/wiki/Hadamard_product_(matrices)) (i.e., elementwise multiplication) of the corresponding prior and likelihood row vectors `i` from each matrix.

Follow these steps to build the posterior as a function of the brain encoded stimulus $\tilde x$:
* For each row of the prior and likelihood (i.e. each possible brain encoding $\tilde x$), fill in the posterior matrix so that every row of the posterior array represents the posterior density for a different brain encode  $\tilde x$.

**Try to implement a vectorial operation to compute the posterior**

In [ ]:
def calculate_posterior_array(prior_array, likelihood_array):
    """
    This function computes the posterior distribution from the prior & the likelihood
    
    Inputs : prior_array (numpy array) is the prior distribution
             likelihood_array (numpy array) is the likelihood distribution
             For both these arrays, each line correspond to a different input stimulus
    Outputs : posterior_array (numpy array) that contains the posterior distribution for the different
              input stimulus (each line corresponds to a different input)
    """

    ###########################
    ### your code goes here ###
    ###########################

    return posterior_array

posterior_array = calculate_posterior_array(prior_array, likelihood_array)

    ###########################
    ### your code goes here ###
    ###########################


               

Click [here](https://github.com/fblondiaux/LGBIO2060-2020/blob/master/Solutions/Tp2-Ex5.py) for solution

*Example output:*

<img alt='Solution hint' align='left' width=573 height=416 src=https://raw.githubusercontent.com/fblondiaux/LGBIO2060-2020/master/Solutions/TP2_Posterior.png>

---
# Section 4: Estimating the position $\hat x$


Now that we have a posterior distribution (for each possible brain encoding $\tilde x$) that represents the brain's estimated ball position: $p(x|\tilde x)$, we want to make an estimate (response) of the ball location $\hat x$ using the posterior distribution. This would represent the subject's estimate if their (for us as experimentalist unobservable) brain encoding took on each possible value. 

This effectively encodes the *decision* that a participant would make for a given brain encoding $\tilde x$. In this exercise, we make the assumption that participants take the mean of the posterior (decision rule) as a response estimate for the glasses location (use the function `moments_myfunc()` provided to calculate the mean of the posterior).

Using this knowledge, we will now represent $\hat x$ as a function of the encoded stimulus $\tilde x$. This will result in a 2D binary decision array. To do so, we will scan the posterior matrix (i.e. row-wise), and set the array cell value to 1 at the mean of the row-wise posterior.

**Suggestions**
* For each brain encoding $\tilde x$ (row of the posterior array), calculate the mean of the posterior, and set the corresponding cell of the binary decision array to 1. (e.g., if the mean of the posterior is at position 0, then set the cell with x_column == 0 to 1).
* Plot the matrix using the function `plot_myarray()` already pre-written and commented-out in your script

In [ ]:
def calculate_binary_decision_array(x_points, posterior_array):
    """
    This function computes the decision taken by the participants for every potential decision input
    
    Inputs : x_points (numpy array) is the set of points on which we evaluated the posterior
             posterior_array (numpy array) is the posterior distribution
             
    Outputs : binary_decision_array (numpy array) that contains the decision taken for every potential input stimulus
    """

    binary_decision_array = np.zeros_like(posterior_array)
    ###########################
    ### your code goes here ###
    ###########################


    return binary_decision_array

binary_decision_array = calculate_binary_decision_array(x, posterior_array)
    ###########################
    ### your code goes here ###
    ###########################


Click [here](https://github.com/fblondiaux/LGBIO2060-2020/blob/master/Solutions/Tp2-Ex6.py) for solution

*Example output:*

<img alt='Solution hint' align='left' width=573 height=416 src=https://raw.githubusercontent.com/fblondiaux/LGBIO2060-2020/master/Solutions/TP2_Decision.png>